# Cutting Stock Problem

### General problem:
You have an order come in for $d_i$ pipes of length $i$ for $i=1, \dots, k$.  How can you fill the order while cutting up the fewest number of pipes?


### Specific Example:
A plumber stocks standard lengths of pipe, all of length 19 m. An order arrives for:

- 12 lengths of 4m
- 15 lengths of 5m
- 22 lengths of 6m

How should these lengths be cut from standard stock pipes so as to minimize
the number of standard pipes used?



In [27]:
# Setup Data
L = [4,5,6] # Lengths of cuts of pipes needed
D = [12,15,22] # Demand for lengths of pipes
L_standard = 19 # Standard legnth of pipes that we can cut

d = [4,3,3] # number of times L[i] goes in to L_standard

## First Model - Board Based


In [25]:
# Import Gurobi
import gurobipy as gp
from gurobipy import GRB
import math

# Create a new model
m = gp.Model("Cutting Stock - Board Based")

# Sets
num_lengths = len(L)
J = range(num_lengths) # index for number types of cuts we can do

num_boards_bound = int(sum([math.ceil(D[i]/d[i]) for i in range(num_lengths)])) # upper bound on the total number of boards that we need.



I = range(num_boards_bound)

# Create variables

# If we use board i, then x_i = 1
x = m.addVars(I, vtype=GRB.BINARY, name="x") 

# If we cut board i into 2 lengths of 4 m and 1 lenghth of 5m, then 
# y[i,0] = 2, y[i,1] = 1.
y = m.addVars(I,J, vtype = GRB.INTEGER, name = "y")

# Set objective - total number of boards used
m.setObjective(sum(x[i] for i in I ), GRB.MINIMIZE)

# Demand constraints:
m.addConstrs((sum(y[i,j] for i in I) >= D[j] for j in J)
             , "Demand")

#Feasible Board cuts
m.addConstrs((sum(L[j]*y[i,j] for j in J) <= L_standard for i in I)
             , "Feasible cuts")

# Activate board if it is cut
m.addConstrs((num_boards_bound*x[i] >= y[i,j] for i in I for j in J), "Turn on x[i]")

# Optimize model
m.optimize()

print("Optimal solution")
#for v in m.getVars():
    #print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 63 rows, 60 columns and 180 nonzeros
Model fingerprint: 0x87b2984c
Variable types: 0 continuous, 60 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 15.0000000
Presolve time: 0.00s
Presolved: 63 rows, 60 columns, 180 nonzeros
Variable types: 0 continuous, 60 integer (15 binary)

Root relaxation: objective 7.333333e+00, 92 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    7.33333    0   44   15.00000    7.33333  51.1%     -    0s
     0     0    9.41838    0   37   15.00000    9.41838  37.2%     -    0s
     0     0   11.50221    0   23   15.00000   11.50221  23.3%     -    0s
     0     0   11.50221    0  

# Pattern Based Formulation


In [30]:
# Import Gurobi
import gurobipy as gp
from gurobipy import GRB
import math

# Create a new model
m = gp.Model("Cutting Stock - Board Based")


P = [[0,0,3],[0,1,2],[1,0,2],
     [0,2,1],[1,2,0],[2,2,0],
     [3,1,0],[4,0,0],[1,3,0],
     [3,0,1]]

num_patterns = len(P)

# Sets



I = range(num_patterns)

# Create variables

# If we use PATTERN i 10 times, then x_i = 10
x = m.addVars(I, vtype=GRB.INTEGER, name="x") 


# Set objective - total number of boards used
m.setObjective(sum(x[i] for i in I ), GRB.MINIMIZE)

# Demand constraints:
m.addConstrs((sum(P[i][j]*x[i] for i in I) >= D[j] for j in J)
             , "Demand")


# Optimize model
m.optimize()

print("Optimal solution")
print('Obj: %g' % m.objVal)
for i in I:
    print(f"# of patterns of type {P[i]} used is: {int(x[i].x)}")




Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3 rows, 10 columns and 18 nonzeros
Model fingerprint: 0x34ca604f
Variable types: 0 continuous, 10 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Found heuristic solution: objective 27.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 9 columns, 16 nonzeros
Variable types: 0 continuous, 9 integer (0 binary)

Root relaxation: objective 1.388889e+01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.88889    0    2   27.00000   13.88889  48.6%     -    0s
H    0     0                      14.0000000   13.88889  0.79%     -    0s
     0     0   13.88889    0    2   14.00000   13.88889  0.79%     -    0s

